<a href="https://colab.research.google.com/github/AlexEMG/DeepLabCut/blob/master/examples/Colab_DEMO_mouse_openfield.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepLabCut Toolbox - Colab Demo on topview mouse data
https://github.com/AlexEMG/DeepLabCut

Nath\*, Mathis\* et al. *Using DeepLabCut for markerless3D  pose estimation during behavior across species. Nature Protocols, 2019

(preprint: https://www.biorxiv.org/content/early/2018/11/24/476531.full.pdf)

This notebook demonstrates the necessary steps to use DeepLabCut on our demo data. We provide labeled mouse data from Mathis et al, 2018 Nature Neuroscience.

This demo notebook mostly shows the most simple code to train and evaluate your model, but many of the functions have additional features, so please check out the overview & the protocol paper!

This notebook illustrates how to use the cloud to:

- load demo data
- create a training set
- train a network
- evaluate a network
- analyze a novel video



##First,  let's look at info about the Colab Environment (and seelct a GPU):

First, go to "Runtime" ->"change runtime type"->select "Python3", and then pick, "GPU!"" (GPUs are faster)


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


If you want to use a GPU:

In [1]:
import tensorflow as tf
tf.__version__

'1.14.0'

In [2]:
#let's make sure we see a GPU:
tf.test.gpu_device_name()

#from tensorflow.python.client import device_lib
#device_lib.list_local_devices()

'/device:GPU:0'

## Link your Google Drive (with our demo data):

### First, download this project folder and place into your Google Drive: 
https://github.com/AlexEMG/DeepLabCut/tree/master/examples/openfield-Pranav-2018-10-30

In [3]:
#Now, let's link to your GoogleDrive. Run this cell and follow the authorization instructions:
#(We recommend putting a copy of the github repo in your google drive if you are using the demo "examples")

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#(this will take a few minutes to install all the dependences!)

!pip install deeplabcut

  Using cached https://files.pythonhosted.org/packages/9b/b7/d829de9794567443fbe230a666910d2c5ea3c28a6554f1246ab004583b82/pandas-0.21.0-cp36-cp36m-manylinux1_x86_64.whl
     |████████████████████████████████| 2.1MB 1.4MB/s 
ERROR: plotnine 0.5.1 has requirement pandas>=0.23.4, but you'll have pandas 0.21.0 which is incompatible.
ERROR: mizani 0.5.4 has requirement pandas>=0.23.4, but you'll have pandas 0.21.0 which is incompatible.
ERROR: imgaug 0.2.9 has requirement numpy>=1.15.0, but you'll have numpy 1.14.5 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 6.0.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=0.24.0, but you'll have pandas 0.21.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement six~=1.12.0, but you'll have six 1.11.0 which is incompatible.
ERROR: featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.21.0 which is incompatible.
ERROR: fbprophet 0.5 ha

In [4]:
#These are some Colab specific work-arounds, but they work! (typically not required, as they are installed with "pip install deeplabcut")

!pip install Pillow==4.0.0

from PIL import Image
def register_extension(id, extension): Image.EXTENSION[extension.lower()] = id.upper()
Image.register_extension = register_extension
def register_extensions(id, extensions): 
  for extension in extensions: register_extension(id, extension)
Image.register_extensions = register_extensions

!pip install pandas==0.23.0
!pip install matplotlib==3.0.3

  Using cached https://files.pythonhosted.org/packages/37/e8/b3fbf87b0188d22246678f8cd61e23e31caa1769ebc06f1664e2e5fe8a17/Pillow-4.0.0-cp36-cp36m-manylinux1_x86_64.whl
ERROR: torchvision 0.3.0 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
ERROR: scikit-image 0.14.3 has requirement pillow>=4.3.0, but you'll have pillow 4.0.0 which is incompatible.
ERROR: imgaug 0.2.9 has requirement numpy>=1.15.0, but you'll have numpy 1.14.5 which is incompatible.
ERROR: fastai 1.0.54 has requirement numpy>=1.15, but you'll have numpy 1.14.5 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 6.1.0
    Uninstalling Pillow-6.1.0:
      Successfully uninstalled Pillow-6.1.0


  Using cached https://files.pythonhosted.org/packages/69/ec/8ff0800b8594691759b78a42ccd616f81e7099ee47b167eb9bbd502c02b9/pandas-0.23.0-cp36-cp36m-manylinux1_x86_64.whl
ERROR: plotnine 0.5.1 has requirement pandas>=0.23.4, but you'll have pandas 0.23.0 which is incompatible.
ERROR: mizani 0.5.4 has requirement pandas>=0.23.4, but you'll have pandas 0.23.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 6.0.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=0.24.0, but you'll have pandas 0.23.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement six~=1.12.0, but you'll have six 1.11.0 which is incompatible.
ERROR: fbprophet 0.5 has requirement pandas>=0.23.4, but you'll have pandas 0.23.0 which is incompatible.
ERROR: fastai 1.0.54 has requirement numpy>=1.15, but you'll have numpy 1.14.5 which is incompatible.
ERROR: deeplabcut 2.0.7.1 has requirement pandas==0.21.0, but you'll have pandas 0.2

**(Be sure to click "RESTART RUNTIME" above before moving on !)**

In [ ]:
#GUIs don't work on the cloud, so we will supress wxPython: 
import os
os.environ["DLClight"]="True"
os.environ["Colab"]="True"

In [4]:
#Be sure you have a copy of your project folder in your Google Drive. Then, let's go to your deeplabcut folder!
%cd /content/drive/My Drive/

/content/drive/My Drive


In [5]:
import deeplabcut

#sometimes the backend kernel is defaulting to something other than what we want
#if you see the warnings, you can re-run this cell and they go way, as we re-set the backend! :) 

#possible warning--> UserWarning: This call to matplotlib.use() has no effect because the backend has already
#been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot, or matplotlib.backends is imported for the first time.

Project loaded in colab-mode. Apparently Colab has trouble loading statsmodels, so the smoothing & outlier frame extraction is disabled. Sorry!
DLC loaded in light mode; you cannot use the labeling GUI!


In [6]:
deeplabcut.__version__

'2.0.7.1'

YOU WILL NEED TO EDIT THE PROJECT PATH **in the config.yaml file** TO BE SET TO YOUR GOOGLE DRIVE LINK!


In [ ]:
#create a path variable that links to your google drive copy:
path_config_file = '/content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/config.yaml' #change to yours!

In [8]:
# Loading example data set:
import os
#path_config_file = os.path.join(os.getcwd(),'openfield-Pranav-2018-10-30/config.yaml')
deeplabcut.load_demo_data(path_config_file)

Loaded, now creating training data...
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


## Start training:
This function trains the network for a specific shuffle of the training dataset. 

In [10]:
#let's also change the display and save_iters just in case Colab takes away the GPU... 
#if that happens, you can reload from a saved point. Typically, you want to train to 200,000 + iterations.
#more info and there are more things you can set: https://github.com/AlexEMG/DeepLabCut/blob/master/docs/functionDetails.md#g-train-the-network

deeplabcut.train_network(path_config_file, shuffle=1, displayiters=10,saveiters=100, maxiters=1000)

#this will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end (default, 1.03M iterations) 
#You can set maxiters to a specific stop-point too. 
#Whichever you chose, you will see what looks like an error message when it stops, but it's not an error - don't worry....

W0703 09:43:23.484168 140132108375936 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/deeplabcut/pose_estimation_tensorflow/training.py:69: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['snout', 'leftear', 'rightear', 'tailbase'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_openfieldOct30/openfield_Pranav95shuffle1.mat',
 'dataset_type': 'default',
 'deterministic': False,
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/usr/local/lib/python3.6/dist-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_wei

Starting with standard pose-dataset loader.


W0703 09:43:26.991242 140132108375936 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/deeplabcut/pose_estimation_tensorflow/nnet/pose_net.py:74: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0703 09:43:27.671877 140132108375936 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/deeplabcut/pose_estimation_tensorflow/nnet/pose_net.py:110: The name tf.losses.sigmoid_cross_entropy is deprecated. Please use tf.compat.v1.losses.sigmoid_cross_entropy instead.

W0703 09:43:27.676215 140132108375936 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0703 09:43:27.712626 140132108375936 deprecation.py:323] From /usr/local/lib/python3

Max_iters overwritten as 1000
Display_iters overwritten as 10
Save_iters overwritten as 100
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'weigh_only_present_joints': False, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': '/content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/dlc-models/iteration-0/openfieldOct30-trainset95shuffle1/train/snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'mirror': False, 'deterministic': False, 'crop_pad': 0, 'scoremap_dir': 'test', 'dataset_type': 'default', 'use_gt_segm': False, 'batch_size': 1, 'video': False, 'video_batch': False, 'crop': True, 'cropratio': 0.4, 'minsize': 100, 'leftwidth': 4

iteration: 10 loss: 0.3536 lr: 0.005
iteration: 20 loss: 0.0523 lr: 0.005
iteration: 30 loss: 0.0402 lr: 0.005
iteration: 40 loss: 0.0382 lr: 0.005
iteration: 50 loss: 0.0424 lr: 0.005
iteration: 60 loss: 0.0383 lr: 0.005
iteration: 70 loss: 0.0299 lr: 0.005
iteration: 80 loss: 0.0275 lr: 0.005
iteration: 90 loss: 0.0227 lr: 0.005
iteration: 100 loss: 0.0245 lr: 0.005
iteration: 110 loss: 0.0336 lr: 0.005
iteration: 120 loss: 0.0235 lr: 0.005
iteration: 130 loss: 0.0256 lr: 0.005
iteration: 140 loss: 0.0239 lr: 0.005
iteration: 150 loss: 0.0227 lr: 0.005
iteration: 160 loss: 0.0223 lr: 0.005
iteration: 170 loss: 0.0287 lr: 0.005
iteration: 180 loss: 0.0207 lr: 0.005
iteration: 190 loss: 0.0256 lr: 0.005
iteration: 200 loss: 0.0247 lr: 0.005
iteration: 210 loss: 0.0206 lr: 0.005
iteration: 220 loss: 0.0273 lr: 0.005
iteration: 230 loss: 0.0202 lr: 0.005
iteration: 240 loss: 0.0210 lr: 0.005
iteration: 250 loss: 0.0223 lr: 0.005
iteration: 260 loss: 0.0205 lr: 0.005
iteration: 270 loss: 

The network is now trained and ready to evaluate. Use the function 'evaluate_network' to evaluate the network.


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py", line 1356, in _do_call
    return fn(*args)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py", line 1341, in _run_fn
    options, feed_dict, fetch_list, target_list, run_metadata)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py", line 1429, in _call_tf_sessionrun
    run_metadata)
tensorflow.python.framework.errors_impl.CancelledError: Enqueue operation was cancelled
	 [[{{node fifo_queue_enqueue}}]]

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/deeplabcut/pose_estimation_tensor

We recommend you run this for ~1,000 iterations, just as a demo. This should take around 20 min. Note, that **when you hit "STOP" you will get a KeyInterrupt "error"! No worries! :)**

## Start evaluating:
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [11]:
%matplotlib notebook
deeplabcut.evaluate_network(path_config_file,plotting=True)

# Here you want to see a low pixel error! Of course, it can only be as good as the labeler, so be sure your labels are good!

Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['snout', 'leftear', 'rightear', 'tailbase'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_openfieldOct30/openfield_Pranav95shuffle1.mat',
 'dataset_type': 'default',
 'deterministic': False,
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/usr/local/lib/python3.6/dist-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'max_input_size': 1500,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets/iteration-0/UnaugmentedDataSet_openfieldOct30/Documentation_data-openfield_95shuffle1.pickle',
 'min_input

Running  DeepCut_resnet50_openfieldOct30shuffle1_1000  with # of trainingiterations: 1000


0it [00:00, ?it/s]

Analyzing data...


116it [00:09, 12.15it/s]


Done and results stored for snapshot:  snapshot-1000
Results for 1000  training iterations: 95 1 train error: 5.72 pixels. Test error: 25.27  pixels.
With pcutoff of 0.4  train error: 5.6 pixels. Test error: 25.27 pixels
Thereby, the errors are given by the average distances between the labels by DLC and the scorer.
Plotting...
The network is evaluated and the results are stored in the subdirectory 'evaluation_results'.
If it generalizes well, choose the best model for prediction and update the config file with the appropriate index for the 'snapshotindex'.
Use the function 'analyze_video' to make predictions on new videos.
Otherwise consider retraining the network (see DeepLabCut workflow Fig 2)


**Check the images**:
You can go look in the newly created "evalutaion-results" folder at the images. At around 3500 iterations, the error is ~3 pixels (but this can vary on how your demo data was split for training)

## Start Analyzing videos: 
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

**On the demo data, this should take around ~ 3 min! **

In [12]:
#Enter the list of videos OR a folder path to analyze:
videofile_path = ['/content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/videos/m3v1mp4.mp4'] 
deeplabcut.analyze_videos(path_config_file,videofile_path, videotype='.mp4')

Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['snout', 'leftear', 'rightear', 'tailbase'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_openfieldOct30/openfield_Pranav95shuffle1.mat',
 'dataset_type': 'default',
 'deterministic': False,
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/usr/local/lib/python3.6/dist-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'max_input_size': 1500,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets/iteration-0/UnaugmentedDataSet_openfieldOct30/Documentation_data-openfield_95shuffle1.pickle',
 'min_input

Using snapshot-1000 for model /content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/dlc-models/iteration-0/openfieldOct30-trainset95shuffle1
Starting to analyze %  /content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/videos/m3v1mp4.mp4
Loading  /content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/videos/m3v1mp4.mp4


  0%|          | 0/2330 [00:00<?, ?it/s]

Duration of video [s]:  77.67 , recorded with  30.0 fps!
Overall # of frames:  2330  found with (before cropping) frame dimensions:  640 480
Starting to extract posture


2346it [02:50, 13.74it/s]                          

Saving results in /content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/videos...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract any outlier frames!


'DeepCut_resnet50_openfieldOct30shuffle1_1000'

## Create labeled video:
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [13]:
deeplabcut.create_labeled_video(path_config_file,videofile_path)

  1%|          | 14/2330 [00:00<00:17, 132.37it/s]

Starting %  /content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/videos ['/content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/videos/m3v1mp4.mp4']
Loading  /content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/videos/m3v1mp4.mp4 and data.
False 0 640 0 480
2330
Duration of video [s]:  77.67 , recorded with  30.0 fps!
Overall # of frames:  2330 with cropped frame dimensions:  640 480
Generating frames and creating video.


100%|██████████| 2330/2330 [00:15<00:00, 147.09it/s]


## Plot the trajectories of the analyzed videos:
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [14]:
deeplabcut.plot_trajectories(path_config_file,videofile_path)

/content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/videos/m3v1mp4.mp4
Starting %  /content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/videos ['/content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/videos/m3v1mp4.mp4']
/content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/videos  already exists!
Loading  /content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/videos/m3v1mp4.mp4 and data.
Plots created! Please check the directory "plot-poses" within the video directory
